In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import  pyspark.sql.functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType
from pyspark.sql import Window

In [3]:
transactions_data_path = "../data/transactions.csv"
catalogue_path = "../data/catalogue.json"
test_path = "../data/test_users.json"
ratings_path = "../data/ratings.csv"
bookmarks_path = "../data/bookmarks.csv"

In [4]:
spark = SparkSession.builder \
            .appName("geneRating") \
            .getOrCreate()

### Bookmarks exploration

In [5]:
bm_df = spark.read.csv(bookmarks_path, header=True)

In [6]:
bm_df = bm_df.withColumn("bookmark", F.lit(1))

### Transactions exploration

Загружаем данные по транзакциям

In [7]:
tr_df = spark.read.csv(transactions_data_path, header=True)

### Загружаем каталог

In [8]:
with open(catalogue_path, "r") as f:
    cat_df = json.load(f)

cat_df = pd.DataFrame.from_dict(cat_df, orient='index')

cat_df = cat_df.reset_index()

cat_df.columns = ['element_uid'] + list(cat_df.columns[1:])

cat_df.element_uid = cat_df.element_uid.astype('int64')

In [9]:
catalogue = spark.createDataFrame(cat_df)

### Data preprocessing

Добавляем к транзакциям информацию о фильме и выбрасываем транзакции по фильмам, по которым нет информации.

In [10]:
tr_df.count()

9643012

In [11]:
tr_df.show(5)

+-----------+--------+----------------+------------------+------------+-----------+-------------------+
|element_uid|user_uid|consumption_mode|                ts|watched_time|device_type|device_manufacturer|
+-----------+--------+----------------+------------------+------------+-----------+-------------------+
|       3336|    5177|               S|44305181.218020596|        4282|          0|                 50|
|        481|  593316|               S|44305180.606027626|        2989|          0|                 11|
|       4128|  262355|               S| 44305180.41444582|         833|          0|                 50|
|       6272|   74296|               S|44305180.202782735|        2530|          0|                 99|
|       5543|  340623|               P|44305179.556133375|        6282|          0|                 50|
+-----------+--------+----------------+------------------+------------+-----------+-------------------+
only showing top 5 rows



Будем использовать только информацию о типе фильма и о его длительности

In [10]:
catalogue = catalogue.select(['element_uid', 'type', 'duration'])

In [11]:
df = tr_df.join(catalogue, on='element_uid', how='inner')

Видим, что транзакций по неизвестным фильмам нет

In [12]:
df = df.withColumn("watched_time", F.col("watched_time")/60)

Заметим, что 1% представленного каталога имеет длительность фильма = 0

In [17]:
catalogue.where(catalogue['duration'] == 0).count() / catalogue.count() * 100

1.1176470588235294

Заменим эту длительность на 75%й квантиль времени просмотра этих фильмов

Так как неизвестно, сколько частей в многосерийном фильме и сериале, для каждого такого фильма и сериала посчитаем 75-квантиль и возьмем его за длительность фильма/сериала. Предварительно переведем длительность просмотра в минуты.

In [13]:
quantiles = df.filter((df['type'] == "multipart_movie") | (df['type'] == "series") | (df['duration'] == 0))\
    .groupBy('element_uid').agg(F.expr("percentile_approx(watched_time, 0.75)").alias('true_duration'))

In [14]:
df = df.join(quantiles, on="element_uid", how="left")

In [15]:
df = df.withColumn("true_duration", F.coalesce("true_duration", "duration"))

Добавим колонку с информацией о том, какую часть фильма посмотрел пользователь, предварительно удалив записи, по которым неизвестна длительность просмотра (около 100к)

In [16]:
df = df.where(df['watched_time'].isNotNull())\
    .withColumn("watch_part", F.col('watched_time') / F.col('true_duration'))

Посмотрим, какую часть всех фильмов составляют фильмы с watch_part > 1

In [17]:
multiwatch_films = df.where((df['type'] == 'movie') & (df['watch_part'] > 1))
films = df.where(df['type'] == 'movie')

print(f"Multiwatch films are {multiwatch_films.count()/films.count() * 100} perc")

Multiwatch films are 28.20763860676843 perc


Считаем, что если длительность просмотра > 1,то пользователь просмотрел фильм/сериал несколько раз. Пока не будем это учитывать и заменим зачение на 1

In [18]:
# multiwatch strategy
df = df.withColumn("true_watch_part", F.when(df['watch_part'] > 12.0, 3.0)\
                                       .when(df['watch_part'] > 1.0, 1.0 + (df['watch_part'] / 6.0))
                                       .otherwise(F.col("watch_part")))

### Ratings

In [20]:
ratings_df = spark.read.csv(ratings_path, header=True)

In [21]:
ratings_df.show(5)

+--------+-----------+------+------------------+
|user_uid|element_uid|rating|                ts|
+--------+-----------+------+------------------+
|  571252|       1364|    10| 44305174.26309871|
|   63140|       3037|    10|44305139.282818206|
|  443817|       4363|     8| 44305136.20584908|
|  359870|       1364|    10| 44305063.00637321|
|  359870|       3578|     9| 44305060.73913281|
+--------+-----------+------+------------------+
only showing top 5 rows



In [22]:
df = df.join(ratings_df.select(['user_uid', 'element_uid', 'rating']),
             on=['user_uid', 'element_uid'], how='left')\
        .join(bm_df.select(['user_uid', 'element_uid', 'bookmark']),
             on=['user_uid', 'element_uid'], how='left')

In [23]:
df = df.fillna(0, subset=["bookmark"])

In [24]:
df = df.select([
    'user_uid',
    'element_uid',
    'consumption_mode',
    'ts',
    'type',
    'true_duration',
    'true_watch_part',
    'rating',
    'bookmark'
])

### StepDan evristic

In [25]:
df = df.withColumn('rating', F.col('rating') + 1)

In [26]:
user_rating_count = df.filter(df['rating'].isNotNull())\
                        .groupBy('user_uid').agg(F.count('rating').alias("rate_count"),
                                                 F.mean('rating').alias("rate_mean"))

In [27]:
user_rating_count.show(5)

+--------+----------+-----------------+
|user_uid|rate_count|        rate_mean|
+--------+----------+-----------------+
|  190265|        17|8.705882352941176|
|  214945|         9|9.333333333333334|
|  552277|         3|9.333333333333334|
|   59652|        11|9.272727272727273|
|   70962|         2|             10.0|
+--------+----------+-----------------+
only showing top 5 rows



In [28]:
tmp_df = df.join(user_rating_count, on='user_uid', how='left')

In [29]:
user_implicit = df.groupBy('user_uid').agg(F.mean('true_watch_part').alias('mean_implicit'))

In [30]:
tmp_df = tmp_df.join(user_implicit, on='user_uid', how='left')

In [31]:
tmp_df = tmp_df.withColumn('rate', F.when((tmp_df['rate_count'] >= 5) & (tmp_df['rating'].isNotNull()),
                                         F.col('rating')/F.col('rate_mean') * F.col('mean_implicit'))\
                                   .otherwise(F.col('true_watch_part'))
                                   
                          )

In [30]:
# cut off
#tmp_df = tmp_df.withColumn('rate', F.when(tmp_df['rate'] > 1, 1).otherwise(F.col('rate')))

In [33]:
tmp_df.where(tmp_df['rate'].isNull()).show()

+--------+-----------+----------------+---+----+-------------+---------------+------+--------+----------+---------+-------------+----+
|user_uid|element_uid|consumption_mode| ts|type|true_duration|true_watch_part|rating|bookmark|rate_count|rate_mean|mean_implicit|rate|
+--------+-----------+----------------+---+----+-------------+---------------+------+--------+----------+---------+-------------+----+
+--------+-----------+----------------+---+----+-------------+---------------+------+--------+----------+---------+-------------+----+



In [65]:
tmp_df.describe('rate').show()

+-------+------------------+
|summary|              rate|
+-------+------------------+
|  count|           9643012|
|   mean|0.6197020692138338|
| stddev|0.4004742401596306|
|    min|               0.0|
|    max|0.9954818185852431|
+-------+------------------+



# parquet write

In [19]:
#запись в файл
default_cols = ['element_uid', 'user_uid', 'consumption_mode', 'ts', 'device_type', 'device_manufacturer', 'type',
              'true_duration', 'true_watch_part']

def write_to_prq(df, default_cols, path="../data/ratings_prq"):
    df.select(default_cols)\
        .repartition(100)\
        .write.parquet(path)

In [34]:
!rm -rdf "../data/StepDan_ratings_prq"

In [35]:
#запись в файл
need_cols = ['element_uid', 'user_uid', 'ts', 'rate']

write_to_prq(tmp_df, need_cols, "../data/StepDan_ratings_prq")

## Multiwatch

In [71]:
multiwatch_films.describe('watch_part').show()

+-------+------------------+
|summary|        watch_part|
+-------+------------------+
|  count|           2382961|
|   mean|1.4906162821732925|
| stddev| 1.750655670823487|
|    min|1.0000793650793651|
|    max|           509.647|
+-------+------------------+



In [72]:
watch_times = multiwatch_films\
                .withColumn("watch_part_int", multiwatch_films["watch_part"].cast(IntegerType()))\
              .select('watch_part_int')


In [80]:
watch_times.groupBy('watch_part_int').agg(F.count('watch_part_int'))\
            .orderBy('count(watch_part_int)', ascending=False)\
            .show()

+--------------+---------------------+
|watch_part_int|count(watch_part_int)|
+--------------+---------------------+
|             1|              2085586|
|             2|               181542|
|             3|                49994|
|             4|                22066|
|             5|                12255|
|             6|                 7707|
|             7|                 4977|
|             8|                 4012|
|             9|                 2854|
|            10|                 1956|
|            11|                 1523|
|            12|                 1241|
|            13|                  952|
|            14|                  801|
|            15|                  687|
|            16|                  549|
|            17|                  504|
|            18|                  446|
|            19|                  381|
|            20|                  328|
+--------------+---------------------+
only showing top 20 rows

